Data_Engineering_TIL_(20190629)

study program : https://www.fastcampus.co.kr/dev_camp_devb


#### [학습목표]


- Web Server에 웹 프로젝트(HTML, CSS, JS) 배포


- AWS ELB, Auto scaling group을 통한 다중 웹서버 환경 구축


- 운영서버 아키텍처 이해

####  [학습기록]

#### STEP 1) EC2를 이용한 웹서버 구현

STEP 1-1) AWS EC2 서비스 콘솔에서 EC2를 먼저 생성한다. 그리고 생성이 완료되면 EC2의 public IP를 확인하여 터미널로 아래 그림과 같은 명령어를 입력하여 생성한 EC2에 접속한다.

** 주의사항 :  EC2 생성 시 보안그룹에서 80 포트를 반드시 개방할 것

STEP 1-2) 아래와 같은 명령어를 입력하여 Nginx를 설치한다.

`sudo su`

`amazon-linux-extras install nginx1.12`

<img src="1.png" width="1000" />

STEP 1-3) 아래 그림과 같은 명령어를 입력하여 Nginx를 구동한다.

<img src="2.png" width="1000" />

STEP 1-4) 생성한 EC2의 퍼블릭 아이피주소를 인터넷 브라우저에 입력하여 아래 그림과 같이 Nginx가 정상적으로 구동하는지 확인한다.

<img src="3.png" width="1000" />

STEP 1-5) 미리 구현된 웹프로젝트를 github에서 가져오기 위해 내가 생성한 EC2에 아래와 같은 명령어로 먼저 git을 설치한다.

<img src="4.png" width="1000" />

STEP 1-6) git 설치가 완료되면 아래 그림과 같은 폴더 경로로 이동하여 미리 지정한 github 사이트에서 미리 구현된 프론트엔드 웹프로젝트를 다운받는다.

<img src="5.png" width="1000" />

STEP 1-7) 다운받은 프론트엔드 웹프로젝트가 Nginx 서버위에서 정상적으로 작동하는지 웹브라우저로 직접 접속하여 각각 체크

1) `http://your-ec2-domain/Fastcampus-web-deploy/` 에 접속하여 아래 그림과 같이 전시가 되는지 확인

** 주의사항 : your-ec2-domain은 생성한 EC2의 퍼블릭 DNS를 말하는 것임

<img src="6.png" width="1000" />

2) `http://your-ec2-domain/Fastcampus-web-deploy/page/album` 에 접속하여 아래 그림과 같이 전시가 되는지 확인

<img src="7.png" width="1000" />

3) `http://your-ec2-domain/Fastcampus-web-deploy/page/signin` 에 접속하여 아래 그림과 같이 전시가 되는지 확인

<img src="8.png" width="1000" />

#### STEP 2) Nginx 설정파일을 수정하여 구현한 웹서버의 DNS 주소를 짧게 설정

- 구현한 웹서버의 DNS 주소는 위에서 보는 것과 같이 너무 길다. 사용자가 이렇게 긴주소를 쓰면 짜증날것이다. 이는 주소를 입력하기 번거로운 문제가 발생한다.


- 또한 보안상으로도 이슈가 있기 때문에(프론트엔드 코드에 서버 디렉토리 구조가 외부에 전부 노출되기 때문이다.) 주소 길이를 줄이고 싶다.


- 그래서 우리는 nginx 설정파일을 수정하여 우리가 구현한 서버 접속시 입력할때 필요한 DNS 주소중 `/Fastcampus-web-deploy` 부분을 생략하는 것을 해볼것이다.

STEP 2-1) 아래 그림과 같은 명령어들을 입력하여 nginx.conf 파일을 열어준다.

** nginx.conf = nginx 설정파일


** 명령어 설명

1) `find / -name nginx.conf` : nginx.conf 파일 위치 찾기

2) `cd /etc/nginx` : nginx.conf 파일이 있는 디렉토리로 이동

3) `cp nginx.conf nginx-copy.conf` : nginx.conf 파일을 수정하기 전에 백업을 해둔다. => 설정파일 수정전에는 반드시 백업을 하는 습관이 필요하다.

4) `vi nginx.conf` : vim 에디터로 nginx.conf 파일열기

<img src="9.png" width="1000" />

STEP 2-2) 아래 그림과 같이 빨간색 박스부분처럼 수정하고 저장해서 빠져나온 다음에 `nginx -s reload` 명령어를 입력하여 수정한 설정을 적용하기 위해 nginx를 재구동한다.

`service nginx restart`를 해도 되는데 보통은 `service nginx restart`으로 프로세스를 재구동하는 것을 권장한다. 리로드는 설정파일을 변경해도 엔진엑스는 실시간으로 변경된 것을 반영해주기 때문에 그냥 리로드만 해줘도 되는 것이지만 통상 설정파일을 수정하면 프로세스를 재구동해줘야 한다.

<img src="10.png" width="1000" />

** 참고사항

위의 빨간색 박스 root 

1) Nginx로 들어오는 http 요청에 응답하기 위해 가장 처음으로 찾는 디렉토리다.

2) 우리는 최초에 `/usr/share/nginx/html`라고 설정된 내용을 `/usr/share/nginx/html/Fastcampus-web-deploy`로 변경했다.

STEP 2-3) `http://your-ec2-dns/page/album/`로 접속하여 아래 그림과 같이 정상적으로 작동하는지 확인한다.

** 참고사항

nginx는 디폴트로 `/usr/share/nginx/html/Fastcampus-web-deploy/` 폴더에 있다고 보면 되고 `http://your-ec2-dns/page/album/` 접속 시 `/page/album/` 이 부분부터 nginx는 티렉토리 탐색을 할 것이다. 다시말해서 엔진엑스의 탐색시점이 바뀐것이다.

<img src="11.png" width="1000" />

STEP 2-4) 여기서 더 나아가 `/page` 부분도 생략해서 예를들어 앨범 index에 접근하고 싶으면 `http://your-ec2-dns/album/`로 접속해보자. 

다시말해서 Nginx 설정 파일 내용 중 location의 내용을 바꿔 `http://your-ec2-dns/album/`로 접속했지만 기존에 `http://your-ec2-dns/page/album/` 으로 입력해서 접근한것처럼 해보자

이를 위해 nginx.conf 파일을 다시 열어서 아래 그림과 같이 빨간색 부분을 추가해준다.

<img src="12.png" width="1000" />

다시말해서 서버 접근을 위해 DNS 주소를 입력하는데 입력내용 중에 album이나 signin 문자열이 있으면 엔진엑스는 `/usr/share/nginx/html/Fastcampus-web-deploy/page` 디렉토리부터 탐색하라는 의미이다.

STEP 2-5) 위와 같이 수정을 완료하면 nginx.conf 파일을 저장하고 빠져나와서 `service nginx restart` 명령으로 nginx를 재구동한다.

<img src="13.png" width="1000" />

STEP 2-6)  `http://your-ec2-dns/album/` 로 접속하여 아래 그림과 같이 정상적으로 전시가 되는지 확인한다.

<img src="14.png" width="1000" />

STEP 2-7) 서버운영 간 로드밸런서의 필요성

<img src="14-2.png" width="1000" />

우리는 그래서 n대의 서버 위에 로드벨런서를 달아보고자 한다.

그리고 특정상황에 따라 오토스케일링 그룹을 이용해서 탄력적으로 EC2를 늘렸다 줄였다 해볼것이다. 오토스케일링은 쉽게말해서 똑같은 일을하고 있는 서버를 자동으로 늘려줬다 줄여줬다 하는 것이다.

#### STEP 3) AWS ELB, Auto scaling group을 이용한 다중 웹서버 환경 구축


STEP 3-1) 먼저 아래 그림과 같이  `systemctl enable nginx` 명령어를 리눅스 터미널에서 입력하여 EC2 인스턴스가 생성될때 Nginx 프로세스를 자동으로 실행하게 한다. (systemctl에 nginx를 등록)

<img src="15.png" width="1000" />

STEP 3-2) 구현한 EC2 웹서버를 AMI 빌드를 수행한다. (구현한 EC2를 스냡샷을 찍어서 똑같은 EC2를 쉽게 만들수 있도록 조치)

아래 그림과 같이 EC2 콘솔에서 내가 만든 EC2 웹서버 이미지를 클릭하고 이미지 생성 버튼을 눌러준다.

<img src="16.png" width="1000" />

그리고 아래 그림과 같이 설정하고 이미지 생성을 클릭한다.

<img src="17.png" width="1000" />

그리고 아래 그림과 같이 AMI 콘솔에서 이미지가 정상적으로 생성되었는지 확인한다.

<img src="18.png" width="1000" />

STEP 3-3) EC2를 하나 더 생성하는데 생성 시 우리가 만들어 놓은 AMI 이미지 파일을 아래 그림과 같이 선택하여 진행해준다.

<img src="19.png" width="1000" />

STEP 3-4) EC2 생성 확인

1) EC2 콘솔 상에서 정상적으로 EC2가 생성이 되었는지 확인

<img src="20.png" width="1000" />

2) 새로 생성한 EC2의 퍼블릭 아이피를 이용하여 웹브라우저에 아래 그림의 빨간색 박스처럼 접속했을때 아래 그림과 같이 정상적으로 전시가 되는지 확인

<img src="21.png" width="1000" />

STEP 3-5) 아래 그림과 같이 로드벨런서를 생성해준다.

** 참고사항

참고로 아래 그림중에 '내부 로드밸런스를 생성'을 설정하면 퍼블릭 아이피가 부여되지 않는다. 프라이빗 아이피가 부여된다.

이말이 무슨말이냐면 외부에서는 로드벨런스 하고만 접촉하면 되고 로드벨런스와 거기에 물려있는 서버들은 프라이빗 아이피로 통신해도 충분한 경우가 있다는 말이다.

비정상 임계값은 로드벨런서가 서버를 헬스체크할때 한번 잘못 대답했다고 무조건 이 서버가 죽었다고 판단할 수 없으니까 2이상 준것이다. 연속적으로 두번 받으면 이 서버가 문제가 있다고 로드벨런서는 판단할 것이다.

비정상으로 판단된 서버가 발생하면 운영자는 이 서버를 고칠것이다. 그러면 로드벨런서는 이 서버에 대한 헬스체크를 다시 했을때 아래 그림 기준으로는 10번 연속으로 체크시 200번대의 정상 신호를 대답해주면 다시 이 서버에 다시 부하를 보내겠다는 것이다.

'교차영역 로드벨런싱 활성화'를 체크하면 AWS에서 내부적으로 운영하고 있는 곳에 내 로드벨런싱을 n대 운영해주겠다는 의미이다. 만약의 상황을 대비해서..

<img src="22.png" width="1000" />

위에까지하면 클래식 로드밸런스를 만들고, 헬스체크 룰을 지정하였으며, 해당로드벨런서를 리스너로 구성한 다음에 필요한 인스턴스들을 물려주었다.

STEP 3-6) 로드벨런서 정상구동 확인

로드벨런서가 생성이 완료되면 아래 그림과 같이 로드벨런서 DNS 주소를 확인하고 웹브라우저에서 직접 접속하여 아래 그림과 같이 정상적으로 전시가 되는지 확인한다.

<img src="28.png" width="1000" />

STEP 3-7) 오토스케일링 그룹 생성

1) 먼저 시작구성 생성 

<img src="31.png" width="1000" />

2) `이 시작 구성을 사용하여 Auto Scaling 그룹 생성` 버튼을 누르면 곧이어 바로 오토스케일링 그룹생성 메뉴가 전시된다. 그래서 아래와 같이 설정하고 생성해준다.

먼저 다음 화면이 나오면 아래 그림과 같이 설정해준다.

<img src="36.png" width="1000" />

3) 그 다음에 조정정책 등을 아래 그림과 같이 구성해준다.

우리는 특정서버가 cpu 사용량이 70%가 넘으면 경보를 울리고 오토스케일링을 해주는 것을 구현하고자 한다.

그래서 먼저 서버의 cpu 사용량이 70%가 넘었을때 3개까지 설정하는 경보를 추가한다. (그룹 크기 증가)

중간에 그룹 크기 증가 및 감소 설정할때 알림 보낼 대상에 주제를 생성하고 나의 이메일을 입력해준다. 그 다음에 내 이메일로 들어가서 AWS 계정 인증을 해준다.

그 다음에는 서버의 cpu 사용량이 30% 이하로 떨어졌을때 다시 1개로 줄여주는 경보를 설정한다. (그룹 크기 감소)

이렇게 설정하면 CPU 사용량에 따른 경보가 울릴경우 내 이메일로 실시간 알림을 줄 것이다.

그래서 아래와 같이 생성하면 가장 아래 그림처럼 오토스케일링 그룹이 잘 생성이 되었는지 확인해준다.

<img src="38.png" width="1000" />

** 참고사항

위와 같이 그룹 크기에 따른 경보 생성 시 '주제 생성'을 내 이메일로 설정할경우 내 이메일로 인증하라는 메일이 아래 그림과 같이 오는데 이것을 승인 해줘야 정상적으로 경보알림을 받을 수 있다.

<img src="39.png" width="1000" />

STEP 3-8) 오토스케일링 그룹 정상작동 여부 테스트

서버에 무한루프 코드를 부여하여 CPU 사용량을 강제로 100%로 만들어줬을 경우 우리가 의도한대로 오토스케일링을 통해 3대가 정상적으로 생성되는지 확인

먼저 오토스케일링 그룹의 EC2 인스턴스를 아래 그림과 같이 선택하여 퍼블릭 아이피 주소를 확인한다.

<img src="40.png" width="1000" />

알아낸 아이피로 서버에 접속하여 아래 그림과 같이 코드를 입력해주면 해당 인스턴스의 cpu가 100%로 올라가고, 등록한 이메일에 경보메세지가 오면서 오토스케일링이 구동될 것이다.

다시말해서 서비스에 사용자가 늘어나서 트래픽이 증가했다고 가정하고, CPU 사용률을 강제로 올리기 위해 yes > /dev/null 명령을 입력한다. 그러면 약 5~7분 정도 기다리면 알림과 함께 새로운 EC2 인스턴스가 생성된다. 로드밸런서 탭에서 로드밸런서와 새로 생성된 EC2 인스턴스가 물려있는 지 확인한다.

<img src="41.png" width="1000" />

** 기타 실습 간 참고사항

1) Sticky session 설정 이란

- ELB의 옵션으로 사용된다. ELB는 기본적으로 라운드로빈 방식으로 트래픽을 분산 하는데, 이를 쿠키 또는 세션을 사용하여 트래픽을 분산하는 기능이다. 다시말해서 특정 사용자가 접속을 시도 했을때 처음 접속된 서버로 계속해서 접속되도록 트래픽을 처리하는 방식이다.


- AWS Loadbalancer의 경우 traffic 에 따라서 인스턴스에 http request를 분산하여 전달 하기 때문에 A 서버는 session 정보를 가지고 있지만, B 서버는 가지고 있지 않을 수 있는 상황이 발생한다. 이런 경우 로그인 해서 새로고침을 했더니 로그아웃이 되어 버릴 수도 있다. 어떻게 해결해야 하냐면 쉽게 말해,  sticky하게 해결한다. 만약 a라는 사용자가 x라는 서버에서 응답을 받았다고 하면 x라는 서버에서 계속 응답을 주는 것이다. 


- Sticky session 필요성 : 로그인 세션을 유지하기 위해서 사용한다.

2) AWS Route53을 통해 로드밸런서 DNS와 퍼블릭 도메인(도메인이 있을 경우에만) 연결 관련 학습자료 :  https://opentutorials.org/course/608/3012

내가 만약에 퍼블릭 도메인이 있다면, 나의 퍼블릭 도메인의 네임서버와 내가 운영하고 있는 서버의 ip를 매칭시켜주면 퍼블릭 도메인으로 접속해도 똑같이 내가 운영하고 있는 서버로 접속할 수 있다.

3) 데브옵스 관점에서 git의 활용방안

깃이라는 얘는 수많은 브랜치를 만들수 있고, 아래 그림과 같이 각각 역할에 맞는 브랜치를 분류할 수 있다. 

<img src="42.png" width="1000" />

이 브랜치 별로 코드들에 대해 버전을 관리할 수 있다. 이말은 특정 브랜치에 올라와 있는 코드는 앞으로 테스트를 진행해야 할 코드일 것이고, 다른 브랜치에 올라와 있는 코드들은 테스트가 끝난 버전의 코드가 될 수도 있다. 아니면 어떤 브랜치에는 운영환경에서 돌고 있는 안정적인 코드일 수도 있다. 이를 브랜치와 버전관리를 통해서 관리할 수 있다.

예를들어서 일반적으로 마스터 브랜치 같은 경우에는 운영서버에 들어가는 안정적인 코드(테스트 시나리오 다 끝나고 운영테스트도 해본 안정적인 코드)가 탑재되어 있다. 디벨롭 브랜치에는 개발자들이 코드를 막 만들어서 넣고 있을 것이다. release 브랜치는 staging 브랜치라고 생각하면 된다. 이 중간 단계에는 test branch라는게 또 있을것이다. 

디벨롭 브랜치에서 막 개발되고 있는 코드들이 어느정도 되면 엔지니어에게 요청이 들어올 것이다. 우리 이 코드 테스트 시나리오에 태워서 테스트를 한번 해보자. 그러면 디벨롭 브랜치에 있는 코드를 테스트 브랜치로 머지를 해준다. 그러면 테스트 브랜치로 머지가 되는 순간에 자동으로 이 서비스를 돌려보고 검수해주는 솔루션들이 돌고 있을 것이고 자동으로 테스트가 될 것이다. 그리고 이 테스트 시나리오를 오류 없이 잘 통과하면 이 코드에 버전 테그가 붙을 것이다. 예를 들어 버전 0.1 이런식으로. 그 다음에 staging 브랜치에 머지를 한다. staging 서버의 환경은 마스터 브랜치에서 돌고 있는 운영 서버환경과 거의 동일할 것이다. 이 staging 브랜치에서 도는 서버는 운영 브랜치에서 문제 없이 돌아갈 것인지 테스트만 하면 되기 때문에 이 서버가 24시간 돌 필요가 없다. 그래서 테스트 브랜치에서 테스트가 끝나면 스테이징 브랜치에 머지가 되는 순간에 스테이징 서버 환경이 자동으로 띄워지게 된다. 운영서버와 가장 유사한 staging환경에서 진짜 리얼서버에서 테스트를 진행해보는 것이다.

만약에 staging 환경까지 테스트가 문제없이 끝나면 마스터 브랜치에 탑재를 하게 되는데 물론 여기서도 문제가 발생하지 않을것이라는 보장은 없다. 그래서 만약에 마스터 브랜치에 탑재했는데 문제가 발생하면 바로 전버전의 코드를 roll back해서 다시 push를 날려줘 버전을 다시 전버전으로 교체해버리면 된다.